# Automated Data Collection | Alpha Vantage API

## Overview
A script automating the process of collecting, cleaning, and storing data from financial API, Alpha Vantage.

### 1. Set Up Environment: Import appropriate libraries.

In [1]:
import requests
import json
import pandas as pd
import sqlite3

### 2. Collect Data: Define the data retrieval function.

In [2]:
def get_stock_data(symbol, API_KEY):
    base_url = "https://www.alphavantage.co/query?"
    function = "TIME_SERIES_DAILY" # Alterable based on the data desired. 
    output_size = "compact" # Choice b/w "compact" or "full" depending on the amt of historical data desired.
    datatype = "json"
    
    # Construct the API request URL
    api_url = f"{base_url}function={function}&symbol={symbol}&outputsize={output_size}&apikey={API_KEY}&datatype={datatype}"
    
    # Send request to API
    response = requests.get(api_url)
    
    # Parse the generated JSON response
    data = json.loads(response.text)
    
    return data

In [4]:
# Test 1:
# get_stock_data("AAPL", API_KEY)

### 3. Data Cleaning: Clean and pre-process the data to make it suitable for analysis.

In [5]:
def clean_data(raw_data):
    
    # Convert gathered data into a Pandas DataFrame
    data = pd.DataFrame(raw_data['Time Series (Daily)']).T
    
    # Rename columns
    data.columns = ["Open", "High", "Low", "Close", "Volume"]
    
    # Convert index to datetime
    data.index = pd.to_datetime(data.index)
    
    # Convert data to numeric values
    for column in data.columns:
        data[column] = pd.to_numeric(data[column])

    return data

In [6]:
# Test 2
clean_data(get_stock_data("AAPL", API_KEY))

,Open,High,Low,Close,Volume
2023-08-01,196.235,196.730,195.2800,195.605,34727848
2023-07-31,196.060,196.490,195.2600,196.450,38824113
2023-07-28,194.670,196.626,194.1400,195.830,48291443
2023-07-27,196.020,197.200,192.5500,193.220,47460180
2023-07-26,193.670,195.640,193.3200,194.500,47471868
...,...,...,...,...,...
2023-03-15,151.190,153.245,149.9200,152.990,77167866
2023-03-14,151.280,153.400,150.1000,152.590,73695893
2023-03-13,147.805,153.140,147.7000,150.470,84457122
2023-03-10,150.210,150.940,147.6096,148.500,68572400


### 4. Data Storage: Save the cleaned data into a local file or database for future use.

In [7]:
def store_data(data, symbol, db_path):
    conn = sqlite3.connect(db_path)
    
    # Write the data to a sqlite table
    data.to_sql(symbol, conn, if_exists='replace', index=True)

    conn.close()

### 5. Test Run: Apple Stock Data

In [9]:
def main():
    
    # Define the stock symbol and the API key
    symbol = "stock_ticker"
    API_KEY = "API_KEY"
    db_path = "stock_data.db"

    # Collect data
    raw_data = get_stock_data(symbol, API_KEY)

    # Clean data
    data = clean_data(raw_data)

    # Store data
    store_data(data, symbol, db_path)
    
if __name__ == "__main__":
    main()